## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)


### select the data

In [17]:
df_april = spark.read.json("/user/sprenner/training_official_april_hourly_proportion_prefetching.json")

In [18]:
#df_mai_sim.write.csv('/user/sprenner/forest_mai_official_predictions_numTrees300_seed1234_cache01.csv', sep='\t')

In [19]:
df_april.describe().toPandas()

summary  account          accountHash         clientState  \
0   count   777286               777286              777286   
1    mean     None  1.935684591732567E8                None   
2  stddev     None  1.136131724108708E9                None   
3     min  aagaard          -2127257360  CannotAuthenticate   
4     max   zzheng           2143592175      SourceNotFound   

       clientStateHash              count  \
0               777286             775662   
1  -9.74016245191591E8  3681.395477669397   
2  2.621298683008806E8  7425.211531293677   
3          -1922357674                  1   
4           1269532611             199996   

                                                                                                              dataset  \
0                                                                                                              777286   
1                                                                                                                None   
2                                                                                                                None   
3                                                data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456_tid289065_00   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210_tid13765955_00   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0                777286       777286                777286          777286   
1  -1.072936835947129E7         None  -7.552649492442358E8            None   
2  1.2586751291558871E9         None   8.314865501449581E8            None   
3           -2146605558    data10_hi           -2057006107     c1130_m1847   
4            2147407155   mc16_valid            2109319881      x557_m1947   

    dataset_versionHash datatype          datatypeHash eventType  \
0                777286   777286                777286    777286   
1  1.3991617429934412E7     None   4.947060556405943E8      None   
2  1.3045708543551383E9     None  1.1700303292632785E9      None   
3           -2146991447      AOD           -2070622869  download   
4            2146405934      log            2124512252  download   

  eventTypeHash eventVersion      eventVersionHash   fileScope  \
0        777286       777286                777286      777286   
1    5.937808E7         None  -1.992468944589716E9        None   
2           0.0         None  3.0732739647461325E8        None   
3      59378080       1.13.2           -2044360925   data10_hi   
4      59378080       1.15.5            1261702107  mc16_valid   

                             filename          filenameHash  \
0                              777286                777286   
1                                None   -248557.46481861244   
2                                None  1.2402658149710782E9   
3    AOD.05371635._000001.pool.root.1           -2147478094   
4  log.13787275._000072.job.log.tgz.1            2147482617   

       filesize_double                hits                  hostname  \
0               777286              777286                    777286   
1   6.11402028756827E8   2.010655022732945                      None   
2  1.240877180994579E9  2.3375900926295015                      None   
3                304.0                   1    ReCaS-ui-01.cs.infn.it   
4      1.1991446515E10                  40  xenia.nevis.columbia.edu   

            hostnameHash           hour               hourHash  \
0                 777286         777286                 777286   
1  -1.1804844919942595E8           None  -2.5176970566176414E7   
2   1.2286452227051153E9           None    1.221590937689879E9   
3            -2133418459  2018-04-01 00            -2138243719   
4             2129595856  2018-04-30 23             2144044799   

                  id       id_proportion      int_percentage  \
0             777286              579384              579384   
1           3

In [60]:
df_april.createOrReplaceTempView("april")

In [61]:
file_patterns = spark.sql("SELECT from_unixtime(traceTimeentryUnix, 'y-MM-dd HH') AS date, concat_ws(',', collect_set(datasetScope)) AS files \
FROM april \
GROUP BY date \
ORDER BY date")

In [62]:
file_patterns.limit(10).toPandas()

date                                              files
0  2018-04-01 00               data15_13TeV,mc16_13TeV,data16_13TeV
1  2018-04-01 01  data15_13TeV,mc16_13TeV,data16_13TeV,data17_13TeV
2  2018-04-01 02  data15_13TeV,mc16_13TeV,data16_13TeV,data17_13TeV
3  2018-04-01 03               data15_13TeV,mc16_13TeV,data16_13TeV
4  2018-04-01 04               data15_13TeV,mc16_13TeV,data16_13TeV
5  2018-04-01 05               data15_13TeV,mc16_13TeV,data16_13TeV
6  2018-04-01 06               data15_13TeV,mc16_13TeV,data16_13TeV
7  2018-04-01 07               data15_13TeV,mc16_13TeV,data16_13TeV
8  2018-04-01 08               data15_13TeV,mc16_13TeV,data16_13TeV
9  2018-04-01 09  data15_13TeV,mc16_13TeV,data16_13TeV,data17_13TeV

In [63]:
file_patterns_split = file_patterns.select(col("date").alias("id"), split(col("files"), ",\s*").alias("items"))

In [64]:
file_patterns_split.limit(10).toPandas()

id                                                   items
0  2018-04-01 00                [data15_13TeV, mc16_13TeV, data16_13TeV]
1  2018-04-01 01  [data15_13TeV, mc16_13TeV, data16_13TeV, data17_13TeV]
2  2018-04-01 02  [data15_13TeV, mc16_13TeV, data16_13TeV, data17_13TeV]
3  2018-04-01 03                [data15_13TeV, mc16_13TeV, data16_13TeV]
4  2018-04-01 04                [data15_13TeV, mc16_13TeV, data16_13TeV]
5  2018-04-01 05                [data15_13TeV, mc16_13TeV, data16_13TeV]
6  2018-04-01 06                [data15_13TeV, mc16_13TeV, data16_13TeV]
7  2018-04-01 07                [data15_13TeV, mc16_13TeV, data16_13TeV]
8  2018-04-01 08                [data15_13TeV, mc16_13TeV, data16_13TeV]
9  2018-04-01 09  [data15_13TeV, mc16_13TeV, data16_13TeV, data17_13TeV]

In [65]:
from pyspark.ml.fpm import FPGrowth

df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

train_data, test_data = file_patterns_split.randomSplit([0.8, 0.2], seed=1234)

fpGrowth = FPGrowth(minSupport=0.1, minConfidence=0.5)
model = fpGrowth.fit(train_data)

In [66]:
# Display frequent itemsets.
model.freqItemsets.limit(10).toPandas()

items  freq
0                            [data16_13TeV]   544
1                              [mc16_13TeV]   533
2                [mc16_13TeV, data16_13TeV]   497
3                            [data17_13TeV]   328
4                [data17_13TeV, mc16_13TeV]   291
5  [data17_13TeV, mc16_13TeV, data16_13TeV]   281
6              [data17_13TeV, data16_13TeV]   317
7                              [mc15_13TeV]   309
8                [mc15_13TeV, data17_13TeV]   203
9    [mc15_13TeV, data17_13TeV, mc16_13TeV]   186

In [67]:
# Display generated association rules.
model.associationRules.limit(100).toPandas()

antecedent      consequent  \
0                               [mc15_14TeV, data16_13TeV]  [data17_13TeV]   
1                               [mc15_14TeV, data16_13TeV]    [mc16_13TeV]   
2                               [mc15_14TeV, data16_13TeV]    [mc15_13TeV]   
3                 [mc15_14TeV, data17_13TeV, data16_13TeV]    [mc16_13TeV]   
4                   [mc15_14TeV, mc15_13TeV, data16_13TeV]    [mc16_13TeV]   
5                 [data15_13TeV, mc16_13TeV, data16_13TeV]    [mc15_13TeV]   
6                                 [mc15_14TeV, mc16_13TeV]  [data17_13TeV]   
7                                 [mc15_14TeV, mc16_13TeV]  [data16_13TeV]   
8                                 [mc15_14TeV, mc16_13TeV]    [mc15_13TeV]   
9                               [mc15_13TeV, data16_13TeV]  [data17_13TeV]   
10                              [mc15_13TeV, data16_13TeV]    [mc16_13TeV]   
11                [data15_13TeV, data17_13TeV, mc16_13TeV]  [data16_13TeV]   
12                [data15_13TeV, data17_13TeV, mc16_13TeV]    [mc15_13TeV]   
13                [data15_13TeV, mc15_13TeV, data17_13TeV]    [mc16_13TeV]   
14                [data15_13TeV, mc15_13TeV, data17_13TeV]  [data16_13TeV]   
15                                          [data16_13TeV]    [mc16_13TeV]   
16                                          [data16_13TeV]  [data17_13TeV]   
17                                          [data16_13TeV]    [mc15_13TeV]   
18  [data15_13TeV, data17_13TeV, mc16_13TeV, data16_13TeV]    [mc15_13TeV]   
19    [data15_13TeV, mc15_13TeV, mc16_13TeV, data16_13TeV]  [data17_13TeV]   
20                              [mc15_14TeV, data17_13TeV]    [mc16_13TeV]   
21                              [mc15_14TeV, data17_13TeV]  [data16_13TeV]   
22                              [data17_13TeV, mc16_13TeV]  [data16_13TeV]   
23                              [data17_13TeV, mc16_13TeV]    [mc15_13TeV]   
24                                          [data17_13TeV]    [mc16_13TeV]   
25                                          [data17_13TeV]  [data16_13TeV]   
26                                          [data17_13TeV]    [mc15_13TeV]   
27  [data15_13TeV, mc15_13TeV, data17_13TeV, data16_13TeV]    [mc16_13TeV]   
28                                            [mc16_13TeV]  [data16_13TeV]   
29                                            [mc16_13TeV]  [data17_13TeV]   
..                                                     ...             ...   
46                              [data15_13TeV, mc15_13TeV]  [data16_13TeV]   
47                    [mc15_14TeV, mc15_13TeV, mc16_13TeV]  [data16_13TeV]   
48                  [mc15_13TeV, data17_13TeV, mc16_13TeV]  [data16_13TeV]   
49    [data15_13TeV, mc15_13TeV, data17_13TeV, mc16_13TeV]  [data16_13TeV]   
50                  [mc15_14TeV, mc16_13TeV, data16_13TeV]  [data17_13TeV]   
51                  [mc15_14TeV, mc16_13TeV, data16_13TeV]    [mc15_13TeV]   
52                [data15_13TeV, mc15_13TeV, data16_13TeV]  [data17_13TeV]   
53                [data15_13TeV, mc15_13TeV, data16_13TeV]    [mc16_13TeV]   
54                  [mc15_13TeV, mc16_13TeV, data16_13TeV]  [data17_13TeV]   
55                  [mc15_14TeV, data17_13TeV, mc16_13TeV]  [data16_13TeV]   
56                                            [mc15_13TeV]  [data17_13TeV]   
57                                            [mc15_13TeV]    [mc16_13TeV]   
58                                            [mc15_13TeV]  [data16_13TeV]   
59                            [data15_13TeV, data16_13TeV]    [mc16_13TeV]   
60                            [data15_13TeV, data16_13TeV]    [mc15_13TeV]   
61                            [data15_13TeV, data17_13TeV]    [mc16_13TeV]   
62                            [data15_13TeV, data17_13TeV]  [data16_13TeV]   
63                            [data15_13TeV, data17_13TeV]    [mc15_13TeV]   
64                [data17_13TeV, mc16_13TeV, data16_13TeV]    [mc15_13TeV]   
65                  [data15_13TeV, mc15_13TeV, mc16_13TeV]  [data17_13

In [68]:
test_data2 = spark.sql("SELECT account, concat_ws(',', collect_set(uuid)) AS files \
FROM april \
GROUP BY account")
test_data2 = test_data2.select(col("account").alias("id"), split(col("files"), ",\s*").alias("items"))

#test_data = test_data.select(split(col("uuid"), ",\s*").alias("items"))

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(test_data).orderBy(["prediction"], ascending=[0]).limit(100).toPandas()

#dataset: 0 Predictions - only 3 frequent itemsets with this confidence limit etc
#join_key_april: same wie dataset
#filename (without Scope, even more general): 0 Predictions - not even frequent itemsets or rules derived
#datasetScope: 76 rules, 97 predictions

id  \
0   2018-04-11 02   
1   2018-04-17 08   
2   2018-04-26 05   
3   2018-04-10 05   
4   2018-04-25 07   
5   2018-04-02 06   
6   2018-04-18 07   
7   2018-04-25 05   
8   2018-04-08 09   
9   2018-04-06 02   
10  2018-04-08 12   
11  2018-04-21 05   
12  2018-04-07 01   
13  2018-04-07 02   
14  2018-04-15 15   
15  2018-04-01 16   
16  2018-04-07 08   
17  2018-04-02 00   
18  2018-04-01 04   
19  2018-04-01 00   
20  2018-04-18 08   
21  2018-04-21 03   
22  2018-04-21 08   
23  2018-04-28 04   
24  2018-04-22 02   
25  2018-04-27 05   
26  2018-04-22 08   
27  2018-04-18 21   
28  2018-04-27 06   
29  2018-04-13 18   
..            ...   
70  2018-04-29 23   
71  2018-04-13 08   
72  2018-04-04 10   
73  2018-04-06 14   
74  2018-04-03 18   
75  2018-04-05 09   
76  2018-04-05 16   
77  2018-04-12 23   
78  2018-04-12 19   
79  2018-04-06 16   
80  2018-04-16 17   
81  2018-04-08 18   
82  2018-04-13 06   
83  2018-04-09 11   
84  2018-04-16 13   
85  2018-04-10 16   
86  2018-04-16 03   
87  2018-04-15 23   
88  2018-04-07 18   
89  2018-04-07 16   
90  2018-04-30 02   
91  2018-04-29 21   
92  2018-04-07 22   
93  2018-04-07 13   
94  2018-04-30 21   
95  2018-04-03 10   
96  2018-04-30 23   
97  2018-04-30 20   
98  2018-04-19 22   
99  2018-04-20 01   

                                                                            items  \
0                                      [data15_13TeV, data16_13TeV, data17_13TeV]   
1                                                    [data16_13TeV, data17_13TeV]   
2                                                    [data16_13TeV, data17_13TeV]   
3                                      [data15_13TeV, data16_13TeV, data17_13TeV]   
4                                        [mc15_14TeV, data16_13TeV, data17_13TeV]   
5                                                    [data15_13TeV, data16_13TeV]   
6                                        [mc15_13TeV, data16_13TeV, data17_13TeV]   
7                                        [mc15_13TeV, data16_13TeV, data17_13TeV]   
8                                        [data15_13TeV, mc16_13TeV, data16_13TeV]   
9                                        [data15_13TeV, mc16_13TeV, data16_13TeV]   
10                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
11                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
12                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
13                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
14                         [data15_13TeV, mc16_13TeV, data16_13TeV, mc15_pPb8TeV]   
15                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
16                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
17                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
18                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
19                                       [data15_13TeV, mc16_13TeV, data16_13TeV]   
20                                       [mc16_13TeV, data16_13TeV, data17_13TeV]   
21                         [mc16_5p44TeV, mc16_13TeV, data16_13TeV, data17_13TeV]   
22                                       [mc16_13TeV, data16_13TeV, data17_13TeV]   
23                                       [mc16_13TeV, data16_13TeV, data17_13TeV]   
24                                       [mc16_13TeV, data16_13TeV, data17_13TeV]   
25             [mc16_13TeV, mc15_valid, data16_13TeV, data17_13TeV, data18_13TeV]   
26                                       [mc16_13TeV, data16_13TeV, data17_13TeV]   
27                                       [mc16_13TeV, data16_13TeV, data17_13TeV]   
28  [mc16_13TeV, mc15_valid, data16_13TeV, data17_13TeV, data18_13TeV, mc16_5TeV]   
29                         [data15_13TeV, mc16_13TeV, data16_13TeV, data17_13TeV]   
..                                                                           

In [69]:
test_data.count()

139